## Import packages

Noet that `rasterio` and `geopandas` need to be installed in the current Python environment for this notebook to work.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.measure import regionprops, regionprops_table
from keras.utils import load_img
from keras.saving import load_model
from importlib import reload
import segmenteverygrain as seg
from segment_anything import sam_model_registry, SamPredictor
from tqdm import trange, tqdm
%matplotlib qt

## Load models

In [3]:
model = load_model("seg_model.keras", custom_objects={'weighted_crossentropy': seg.weighted_crossentropy})

# the SAM model checkpoints can be downloaded from: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](checkpoint="/Users/zoltan/Dropbox/Segmentation/sam_vit_h_4b8939.pth")

2024-12-18 11:56:31.978965: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-12-18 11:56:31.979033: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 96.00 GB
2024-12-18 11:56:31.979042: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 36.00 GB
2024-12-18 11:56:31.979092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-18 11:56:31.979145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (t

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

In [4]:
fname = '/Users/zoltan/Dropbox/Segmentation/Kalinda_Roberts_project/RI_T01_Grid_65.tif'
image = np.array(load_img(fname))
image_pred = seg.predict_image(image, model, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, coords = seg.label_grains(image, image_pred, dbs_max_dist=10.0) # Unet prediction

# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, image, image_pred, 
            coords, labels, min_area=50.0, plot_image=True, remove_edge_grains=False, remove_large_objects=False)

# big_im = np.array(load_img(fname))
# big_im_pred = seg.predict_big_image(big_im, model, I=256)
# # decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
# labels, grains, coords = seg.label_grains(big_im, big_im_pred, dbs_max_dist=10.0)
# all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, big_im, big_im_pred, coords, labels, min_area=50.0)

segmenting image tiles...


100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  5.67it/s]


creating masks using SAM...


100%|███████████████████████████████████████| 1301/1301 [00:49<00:00, 26.41it/s]


finding overlapping polygons...


1293it [00:03, 365.44it/s]


finding best polygons...


100%|█████████████████████████████████████████| 393/393 [00:08<00:00, 49.02it/s]


creating labeled image...


100%|████████████████████████████████████████| 428/428 [00:01<00:00, 381.02it/s]


## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [5]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [6]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [9]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 427/427 [00:00<00:00, 2306.76it/s]
427it [00:00, 3641.94it/s]


Plot the updated set of grains:

In [19]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(image)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap='Paired')
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0]);

100%|████████████████████████████████████████| 427/427 [00:01<00:00, 324.71it/s]


## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [20]:
reload(seg)
predictor = SamPredictor(sam)
predictor.set_image(image) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, image, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))

In [21]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [22]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 453/453 [00:00<00:00, 2451.27it/s]
453it [00:00, 3551.44it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [52]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))

number of pixels: 193.11


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [57]:
n_of_units = 500 # centimeters in the case of 'IMG_5208_image.png'
units_per_pixel = n_of_units/193.11 # length of scale bar in pixels

In [58]:
props = regionprops_table(labels.astype('int'), intensity_image = big_im, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [59]:
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,15016.838126,20.546875,945.926786,168.641397,118.488355,1.139348,483.430889,255.0,255.0,255.0,218.377232,212.426339,219.841071,60.0,43.0,13.0
1,2,10176.589408,339.316206,1820.215415,144.012838,92.730107,-1.510110,397.175106,255.0,255.0,255.0,183.735178,177.980237,181.127141,27.0,47.0,38.0
2,3,17182.212552,451.810769,15.046430,231.092918,99.993859,-0.021373,573.868806,238.0,233.0,255.0,205.172844,203.001171,209.460008,62.0,64.0,54.0
3,4,11812.352133,514.209989,629.208286,149.117158,102.839337,0.292422,422.546630,255.0,255.0,255.0,219.042565,214.907491,220.919410,40.0,77.0,66.0
4,5,10712.905056,551.856696,465.237797,177.367748,78.752628,-1.112341,441.483269,255.0,249.0,255.0,204.268461,198.521902,202.667710,53.0,68.0,58.0


In [60]:
# plot histogram of grain axis lengths
# note that input data needs to be in milimeters
fig, ax = seg.plot_histogram_of_axis_lengths(grain_data['major_axis_length'], grain_data['minor_axis_length'], binsize=0.2, xlimits=[0.25, 4])

## Save mask and image to PNG files

In [486]:
dirname = '/Users/zoltan/Dropbox/Segmentation/images/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True

## Convert polygon row, col coordinates to projected coordinates and save them to shapefile

In [24]:
import rasterio
dataset = rasterio.open(fname)

In [25]:
dataset.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 1113,
 'height': 1112,
 'count': 4,
 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 19N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-69],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32619"]]'),
 'transform': Affine(0.0017999999999882017, 0.0, 339041.9403932594,
        0.0, -0.0018000000000683416, 4686358.8424565075)}

In [26]:
# convert polygon coordinates from row, col to UTM
from shapely.geometry import Polygon
projected_polys = []
for grain in all_grains:
    x, y = rasterio.transform.xy(dataset.transform, grain.exterior.xy[1], grain.exterior.xy[0])
    poly = Polygon(np.vstack((x, y)).T)
    projected_polys.append(poly)

In [28]:
# create geopandas dataframe
import geopandas
gdf = geopandas.GeoDataFrame(projected_polys, columns = ['geometry'])
gdf.head()

,geometry
0,"POLYGON ((339043.943 4686358.495, 339043.941 4..."
1,"POLYGON ((339043.824 4686358.479, 339043.822 4..."
2,"POLYGON ((339043.228 4686358.364, 339043.226 4..."
3,"POLYGON ((339043.768 4686358.191, 339043.767 4..."
4,"POLYGON ((339043.036 4686357.764, 339043.034 4..."


In [30]:
# create property dataframe from labeled image
props = regionprops_table(labels.astype('int'), intensity_image = image, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values
grain_data['area'] = grain_data['area'].values
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length
0,1,683.0,182.021962,1097.698389,37.846881,24.424525
1,2,676.0,189.627219,1041.396450,37.309940,23.467704
2,3,410.0,255.880488,719.348780,30.057850,17.937276
3,4,393.0,348.519084,1015.414758,29.249796,19.877529
4,5,354.0,587.093220,607.022599,25.084603,18.253541


In [31]:
# convert centroids from row, col to UTM and add them to geodataframe
centroid_x, centroid_y = rasterio.transform.xy(dataset.transform, grain_data['centroid-0'], 
                                               grain_data['centroid-1'])
gdf['centroid_x'] = centroid_x
gdf['centroid_y'] = centroid_y

In [32]:
# convert grain axis lengths to UTM units
gdf['major_axis_length'] = grain_data['major_axis_length'] * dataset.transform[0]
gdf['minor_axis_length'] = grain_data['minor_axis_length'] * dataset.transform[0]
gdf.head()

,geometry,centroid_x,centroid_y,major_axis_length,minor_axis_length
0,"POLYGON ((339043.943 4686358.495, 339043.941 4...",339043.917150,4.686359e+06,0.068124,0.043964
1,"POLYGON ((339043.824 4686358.479, 339043.822 4...",339043.815807,4.686359e+06,0.067158,0.042242
2,"POLYGON ((339043.228 4686358.364, 339043.226 4...",339043.236121,4.686358e+06,0.054104,0.032287
3,"POLYGON ((339043.768 4686358.191, 339043.767 4...",339043.769040,4.686358e+06,0.052650,0.035780
4,"POLYGON ((339043.036 4686357.764, 339043.034 4...",339043.033934,4.686358e+06,0.045152,0.032856


In [33]:
# check if everything looks good
band1 = dataset.read(1)
band2 = dataset.read(2)
band3 = dataset.read(3)
plt.figure()
plt.imshow(np.stack((band1, band2, band3), axis=2), extent = [dataset.bounds[0], dataset.bounds[2], 
                                         dataset.bounds[1], dataset.bounds[3]])
plt.scatter(gdf['centroid_x'], gdf['centroid_y']);

In [34]:
gdf.crs = dataset.crs # set geodataframe CRS

In [35]:
gdf.to_file("/Users/zoltan/Dropbox/Segmentation/Kalinda_Roberts_project/projected_grains.shp") # write shapefile

/var/folders/wn/31h1q45j6y321jzk91br_d3m0000gn/T/ipykernel_7410/571464815.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("/Users/zoltan/Dropbox/Segmentation/Kalinda_Roberts_project/projected_grains.shp") # write shapefile
/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'major_axis_length' to 'major_axis'
  ogr_write(
/Users/zoltan/miniforge3/envs/segmenteverygrain/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'minor_axis_length' to 'minor_axis'
  ogr_write(


In [36]:
dataset.close()